In [ ]:
#!pip install fiftyone

In [ ]:
!pip install pycocotools

In [ ]:
!python -m pip install --user  opencv-python-headless


In [ ]:
!python -m pip install --user layoutparser # Install the base layoutparser library with
!python -m pip install --user  "layoutparser[layoutmodels]" # Install DL layout model toolkit
!python -m pip install --user "layoutparser[ocr]" # Install OCR toolkit

In [ ]:
!sudo apt-get update
!sudo apt install --yes libgl1 libgl1-mesa-glx libglib2.0-0

In [ ]:

# The directory containing the source images
data_path = "./work/images/"

# The path to the COCO labels JSON file
labels_path = "result_json.json"



In [ ]:
#import fiftyone as fo
# Import the dataset
#dataset = fo.Dataset.from_dir(
#    dataset_type=fo.types.COCODetectionDataset,
#    data_path=data_path,
#    labels_path=labels_path,
#)

In [ ]:
# From  https://github.com/Layout-Parser/layout-parser/blob/main/examples/Customizing%20Layout%20Models%20with%20Label%20Studio%20Annotation/Customizing%20Layout%20Models%20with%20Label%20Studio%20Annotation.ipynb

from pycocotools.coco import COCO
import layoutparser as lp
import random
import cv2

def load_coco_annotations(annotations, coco=None):
    """
    Args:
        annotations (List):
            a list of coco annotaions for the current image
        coco (`optional`, defaults to `False`):
            COCO annotation object instance. If set, this function will
            convert the loaded annotation category ids to category names
            set in COCO.categories
    """
    layout = lp.Layout()

    for ele in annotations:

        x, y, w, h = ele['bbox']

        layout.append(
            lp.TextBlock(
                block = lp.Rectangle(x, y, w+x, h+y),
                type  = ele['category_id'] if coco is None else coco.cats[ele['category_id']]['name'],
                id = ele['id']
            )
        )

    return layout

In [ ]:
COCO_ANNO_PATH = labels_path
COCO_IMG_PATH  = data_path

coco = COCO(COCO_ANNO_PATH)

In [ ]:
for image_id in random.sample(list(coco.imgs.keys()), 2):
    image_info = coco.imgs[image_id]
    annotations = coco.loadAnns(coco.getAnnIds([image_id]))

    image = cv2.imread(f'{COCO_IMG_PATH}/{image_info["file_name"]}')
    layout = load_coco_annotations(annotations, coco)

    viz = lp.draw_box(image[...,::-1], layout)
    display(viz) # show the results